In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append("../")
import copy

import logging
from src.utils import logging_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

In [4]:
from src.dataset import load_dataset, load_relation, fill_template

relation = load_relation(
    # relation_file = "head_of_government.json",
    relation_file="gender_head_of_govt.json",
    default_path="../data",
)

2024-06-06 14:56:29 src.dataset INFO     initialized relation -> "gender head of govt" with 7757 samples


In [5]:
relation.select_icl_examples(num_icl=5)
print(relation[5][0])

In 1884, Luxembourg's Prime Minister was a man
In 1982, India's Prime Minister was a woman
In 1995, Jordan's King was a man
In 2003, Sri Lanka's President was a woman
In 2008, Germany's Chancellor was a woman
In 1991, Sierra Leone's President was a


In [6]:
from src import functional

relation_1971 = functional.filter_samples_by_var(relation=relation, var="1971")

2024-06-06 14:56:29 src.functional INFO     filtered 82 with var=1971, from gender head of govt


In [7]:
relation_1971.few_shot_demonstrations

["In 1971, Colombia's President was a man",
 "In 1971, Israel's Prime Minister was a woman",
 "In 1971, Lesotho's Prime Minister was a man",
 "In 1971, Canada's Prime Minister was a man",
 "In 1971, India's Prime Minister was a woman"]

In [8]:
dataset = load_dataset(
    default_path="../data",
)

2024-06-06 14:56:30 src.dataset INFO     initialized relation -> "head of government" with 8354 samples
2024-06-06 14:56:30 src.dataset INFO     initialized relation -> "gender head of govt" with 7757 samples


## Filter By Model Knowledge


In [9]:
from src.models import ModelandTokenizer

model_name = "meta-llama/Meta-Llama-3-8B"
mt = ModelandTokenizer(model_path=model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-06-06 14:56:34 src.models INFO     loaded model </home/local_arnab/Codes/saved_model_weights/meta-llama/Meta-Llama-3-8B> | size: 15508.516 MB


In [10]:
#! This filtering strategy may cause problem for `gender_head_of_govt`, the model is almost always happy to predict `male` with zero-shot

# relation = load_relation(
#     relation_file = "head_of_government.json",
#     num_icl = 0,                          # initialize with zero-shot
#     default_path="../data",
#     batch_size=500
# )

# all_samples = copy.deepcopy(relation.samples)

# # filter zero-shot model knowledge
# relation = functional.filter_samples_by_model_knowledge(
#     mt = mt,
#     relation = relation,
# )

# relation.properties["num_icl"] = 5
# relation.select_icl_examples(num_icl=5)

# relation.samples = all_samples

# # filter model knowledge with `num_icl` shots
# relation = functional.filter_samples_by_model_knowledge(
#     mt = mt,
#     relation = relation,
# )

In [11]:
relation = load_relation(
    relation_file="head_of_govt.json",
    num_icl=5,  # initialize with 5-shot
    default_path="../data",
    batch_size=500,
)

all_samples = copy.deepcopy(relation.samples)

relation.range_stats

2024-06-06 14:56:34 src.dataset INFO     initialized relation -> "head of government" with 495 samples


'range = |500| count(obj)_min = 1, count(obj)_max = 1'

In [12]:
# # prompt_template = relation.prompt_template
# # relation.prompt_template = relation.prompt_template_zs

# print(relation.prompt_template)
# print(relation[10][0])

In [13]:
relation[1]

("In 1984, Morocco's King was named Mohammed Karim Lamrani\nIn 1947, Netherlands's Prime Minister was named Louis Beel\nIn 1963, Mauritania's President was named Moktar Ould Daddah\nIn 1982, United Kingdom's Prime Minister was named Margaret Thatcher\nIn 2015, Denmark's Prime Minister was named Lars Løkke Rasmussen\nIn 1981, Iceland's Prime Minister was named",
 'Gunnar Thoroddsen')

In [14]:
# Check model knowledge (few-shot examples used to contextualize may be of bad quality -- not known to model)
# performance tends to vary a lot depending on the few-shot examples

relation = functional.filter_samples_by_model_knowledge(
    mt=mt,
    relation=relation,
)

relation.range_stats

2024-06-06 14:56:34 src.functional DEBUG    "head of government" | filtering with meta-llama-3-8b
2024-06-06 14:56:34 src.functional DEBUG    sample.subject='Seychelles'[2003] -> answer='France-Albert René' | predicted = ' France'(0.447) ==> (✓)
2024-06-06 14:56:34 src.functional DEBUG    sample.subject='Iceland'[1981] -> answer='Gunnar Thoroddsen' | predicted = ' Gunn'(0.505) ==> (✓)
2024-06-06 14:56:34 src.functional DEBUG    sample.subject='Ireland'[1989] -> answer='Charles Haughey' | predicted = ' Charles'(0.871) ==> (✓)
2024-06-06 14:56:34 src.functional DEBUG    sample.subject='Denmark'[1951] -> answer='Erik Eriksen' | predicted = ' Erik'(0.925) ==> (✓)
2024-06-06 14:56:34 src.functional DEBUG    sample.subject='Belarus'[1993] -> answer='Stanislav Shushkevich' | predicted = ' Alexander'(0.406) ==> (✗)
2024-06-06 14:56:34 src.functional DEBUG    sample.subject='Morocco'[1997] -> answer='Abdellatif Filali' | predicted = ' Mohammed'(0.796) ==> (✗)
2024-06-06 14:56:34 src.functional 

'range = |365| count(obj)_min = 1, count(obj)_max = 1'

In [15]:
len(relation.range)

365

In [16]:
# # Now, select ICL examples from the filtered set that the model knows
# relation.select_icl_examples(num_icl=5, consider_few_shot_samples=False)
# relation.samples = all_samples

# # Use the known samples to filter by model knowledge again
# relation = functional.filter_samples_by_model_knowledge(
#     mt = mt,
#     relation = relation,
# )
# relation.range_stats

In [17]:
from src.dataset import balance_samples

relation.samples = balance_samples(relation.samples)

relation.range_stats

'range = |365| count(obj)_min = 1, count(obj)_max = 1'